In [4]:
from pyspark.sql import SparkSession
import json

spark = SparkSession \
    .builder \
    .appName("StructuredRealTimeState") \
    .getOrCreate()

import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell'

In [39]:
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql import types as T

jstr1 = '[{"time" : 123456, "name" : "n"}]'
jstr2 = '[{"time" : 33333, "name" : "m"}]'
jstr3 = '[{"time" : 65433212, "name" : "e"}, {"time" : 43221, "name" : "j"}]'
df = spark.createDataFrame([Row(json=jstr1),Row(json=jstr2),Row(json=jstr3)])
# df = spark.createDataFrame([Row(json='[{"time" : 157651},{"time" : 257651}]')])
# df = spark.createDataFrame([Row(json='[{"time" : 65433212}, {"time" : 43221}]')])
df.select(F.col("json").cast("string")).show()
df.collect()

+--------------------+
|                json|
+--------------------+
|[{"time" : 123456...|
|[{"time" : 33333,...|
|[{"time" : 654332...|
+--------------------+



[Row(json='[{"time" : 123456, "name" : "n"}]'),
 Row(json='[{"time" : 33333, "name" : "m"}]'),
 Row(json='[{"time" : 65433212, "name" : "e"}, {"time" : 43221, "name" : "j"}]')]

In [40]:
schema = T.ArrayType(T.StructType()\
    .add("time", T.TimestampType())\
    .add("name", T.StringType()))
# ddf = df.select(F.from_json(F.col("json").cast("string"), schema).alias("json"))
ddf = df.select(F.from_json(F.col("json").cast("string"), schema).alias("value"))
# ddf = df.select(F.from_json(F.col("json"), F.schema_of_json('[{"time" : 65433212}, {"time" : 43221}]')).alias("json"))
ddf.collect()

[Row(value=[Row(time=123456, name='n')]),
 Row(value=[Row(time=33333, name='m')]),
 Row(value=[Row(time=65433212, name='e'), Row(time=43221, name='j')])]

In [41]:
edf = ddf.select(F.explode("value").alias("new_val"))
edf.collect()
# def flatten(row): id_, items = row; return ((id_, item) for item in items)
# ddf.select(F.explode(F.col("value")).alias("new_val")).rdd.flatMap(flatten).toDF(["time"])

[Row(new_val=Row(time=123456, name='n')),
 Row(new_val=Row(time=33333, name='m')),
 Row(new_val=Row(time=65433212, name='e')),
 Row(new_val=Row(time=43221, name='j'))]

In [43]:
edf.select("new_val.*").show()

+--------+----+
|    time|name|
+--------+----+
|  123456|   n|
|   33333|   m|
|65433212|   e|
|   43221|   j|
+--------+----+



In [55]:
from datetime import datetime
import time

d = datetime.now().timestamp()

In [62]:
d = 15770499787
type(d)

int

In [63]:
datetime.fromtimestamp(d)

datetime.datetime(2469, 9, 30, 0, 23, 7)

TypeError: descriptor 'bit_length' of 'int' object needs an argument